In [5]:
# fetching data set from kaggle

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import STOPWORDS
import re
import string
df = pd.read_csv('./spamhamdata.csv',delimiter='\t') # tabs seperated values

In [6]:
def clean(text):
    text = str(text).lower()
    text = re.sub("\[.*?\]",' ',text)
    text = re.sub('https?://\S+|www\.\S+','',text)
    text = re.sub('<.*?>+','', text)
    text = re.sub("[%s]" % string.punctuation, '', text)
    text = re.sub('\w*\d\w*','',text)
    text = " ".join(i for i in text.split() if i not in STOPWORDS)
    return text

In [7]:
df['Text'] = df['Text'].apply(clean)

In [8]:
tf_idf = TfidfVectorizer()

In [9]:
text_matrix = tf_idf.fit_transform(df['Text'])

In [10]:
df['Spam'] = df['Spam'].map({'ham':0,'spam':1})

In [11]:
y = df['Spam']

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


xtrain, xtest, ytrain, ytest = train_test_split(text_matrix.toarray(), y,test_size=0.2)
model = GaussianNB()
model.fit(xtrain, ytrain)
print("The accuracy of this model is: {}".format(model.score(xtest, ytest)))

The accuracy of this model is: 0.8726457399103139


In [13]:
from sklearn.metrics._classification import classification_report

print(classification_report(ytest, model.predict(xtest)))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       971
           1       0.50      0.89      0.64       144

    accuracy                           0.87      1115
   macro avg       0.74      0.88      0.78      1115
weighted avg       0.92      0.87      0.89      1115



In [14]:
def main():
    inp = input("Enter the message: ")
    inp = clean(inp)
    transformed = tf_idf.transform([inp]).toarray()
    ans = model.predict(transformed)
    map = {0: "ham", 1: "spam"}
    print("The message is: '{}'".format(map[ans[0]]))

In [15]:
main()

Enter the message: *Join Above all links to stay updated.**  Internship will commence  on 12th April 2024.  Key Guidelines & Dates-  1) LinkedIn Sharing: Update your profile, mention @Coding Samurai, and use #CodingSamurai to showcase achievements.  2) Originality Priority: Avoid plagiarism. Copied work leads to termination and blocks future opportunities.  3) Display Progress: Share task videos on LinkedIn, tag @coding samurai, use #CodingSamurai, and provide task insights.  4) Tech Interns' GitHub/ Behance Dribble for designers: Set up "CODINGSAMURAI" repository. Share link in upcoming task submission email.
The message is: 'ham'
